In [1]:
import sqlite3
import pandas as pd

In [2]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None

In [3]:
conn = sqlite3.connect('parch-and-posey.db')

In [4]:
cursor = conn.cursor()
cursor.execute('''
select * from sqlite_master where type = "table";
''')
columns = [col[0] for col in cursor.description]
data = cursor.fetchall()
cursor.close()

In [5]:
pd.DataFrame(data, columns=columns)

,type,name,tbl_name,rootpage,sql
0,table,web_events,web_events,2,"CREATE TABLE web_events (\tid integer,\taccoun..."
1,table,sales_reps,sales_reps,92,"CREATE TABLE sales_reps (\tid integer,\tname b..."
2,table,region,region,93,"CREATE TABLE region (\tid integer,\tname bpchar)"
3,table,orders,orders,94,"CREATE TABLE orders (\tid integer,\taccount_id..."
4,table,accounts,accounts,221,"CREATE TABLE accounts (\tid integer,\tname bpc..."


Task 1:

Use DATE_TRUNC to pull month level information about the first order ever placed in the orders table.

In [6]:
# way too complex

pd.read_sql_query(sql='''
SELECT strftime('%Y-%m', min_date) AS min_month
FROM (SELECT MIN(occurred_at) AS min_date
FROM orders
ORDER BY 1
LIMIT 1);
''', con=conn)

,min_month
0,2013-12


In [7]:
# better

pd.read_sql_query(sql='''
SELECT strftime('%Y-%m', MIN(occurred_at)) 
FROM orders;
''', con=conn)

,"strftime('%Y-%m', MIN(occurred_at))"
0,2013-12


Solution:

SELECT DATE_TRUNC('month', MIN(occurred_at)) 
FROM orders;

Task 2:

Use the result of the previous query to find only the orders that took place in the same month and year as the first order, and then pull the average for each type of paper qty in this month.

In [8]:
pd.read_sql_query(sql='''
SELECT AVG(standard_qty) standard, AVG(gloss_qty) gloss, AVG(poster_qty) poster, SUM(total_amt_usd) total_usd FROM orders WHERE
strftime('%Y-%m', occurred_at) = 
(SELECT strftime('%Y-%m', MIN(occurred_at)) 
FROM orders);
''', con=conn)

,standard,gloss,poster,total_usd
0,268.222222,208.949495,111.818182,377331.0


Solution:

SELECT AVG(standard_qty) avg_std, AVG(gloss_qty) avg_gls, AVG(poster_qty) avg_pst
FROM orders
WHERE DATE_TRUNC('month', occurred_at) = 
     (SELECT DATE_TRUNC('month', MIN(occurred_at)) FROM orders);

SELECT SUM(total_amt_usd)
FROM orders
WHERE DATE_TRUNC('month', occurred_at) = 
      (SELECT DATE_TRUNC('month', MIN(occurred_at)) FROM orders);